In [ ]:
from ipywidgets import Checkbox, Dropdown, Accordion, VBox
import sklearn.metrics as metrics
from sklearn.calibration import calibration_curve
from spektral.data import Dataset, Graph, DisjointLoader
from spektral.layers import CrystalConv, GlobalAvgPool
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

import copy
import logging
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import random
import sys
import tensorflow as tf
import requests
import progressbar
from os.path import isfile
import time

# Setting up the logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
stdout_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stdout_handler)

In [ ]:
data = pd.read_pickle(r"C:\Users\Barre\Downloads\men.pkl")


In [ ]:
print("Choose Adjacency Matrix:")
adj_matrix = Dropdown(
    options=['normal', 'delaunay', 'dense', 'dense_ap', 'dense_dp'],
    value='normal',
    disabled=False,
)
display(adj_matrix)

In [ ]:
og_data = data

In [ ]:
player_dist = Checkbox(
    value=True,
    description='Player Distance',
    disabled=False
)

speed_diff_matrix = Checkbox(
    value=True,
    description='Speed Difference',
    disabled=False
)

pos_sin_angle = Checkbox(
    value=True,
    description='Positional Sine angle',
    disabled=False
)

pos_cos_angle = Checkbox(
    value=True,
    description='Positional Cosine angle',
    disabled=False
)

vel_sin_angle = Checkbox(
    value=True,
    description='Velocity Sine angle',
    disabled=False
)

vel_cos_angle = Checkbox(
    value=True,
    description='Velocity Cosine angle',
    disabled=False
)

# Add the user selection of edge features in edge_f_box list.
edge_f_box = VBox([player_dist, speed_diff_matrix, pos_sin_angle, pos_cos_angle, vel_sin_angle, vel_cos_angle])

In [ ]:
x_coord = Checkbox(
    value=True,
    description='x coordinate',
    disabled=False
)

y_coord = Checkbox(
    value=True,
    description='y coordinate',
    disabled=False
)

vx = Checkbox(
    value=True,
    description='vX',
    disabled=False
)


vy = Checkbox(
    value=True,
    description='vY',
    disabled=False
)


v = Checkbox(
    value=True,
    description='Velocity',
    disabled=False
)


velocity_angle = Checkbox(
    value=True,
    description='Velocity Angle',
    disabled=False
)

dist_goal = Checkbox(
    value=True,
    description='Distance to Goal',
    disabled=False
)

goal_angle = Checkbox(
    value=True,
    description='Angle with Goal',
    disabled=False
)


dist_ball = Checkbox(
    value=True,
    description='Distance to Ball',
    disabled=False
)

ball_angle = Checkbox(
    value=True,
    description='Angle with Ball',
    disabled=False
)


is_attacking = Checkbox(
    value=True,
    description='Attacking Team Flag',
    disabled=False
)

potential_receiver = Checkbox(
    value=True,
    description='Potential Receiver',
    disabled=False
)

# Add the user selection of node features in node_f_box list.
node_f_box = VBox([x_coord, y_coord, vx, vy, v, velocity_angle, dist_goal, goal_angle, dist_ball, ball_angle, 
                is_attacking, potential_receiver])

In [ ]:
accordion = Accordion(children=[edge_f_box, node_f_box])
accordion.set_title(0, 'Edge Features')
accordion.set_title(1, 'Node Features')
accordion

In [ ]:
class CounterDataset(Dataset):
    """
    Convert raw Graph data to a CounterDataset with Dataset class from the spektral library to include node features, 
    edge features, and the adjacency matrix.
    """
    def __init__(self, **kwargs):
        '''
        Constructor to load parameters.
        '''
        self.data = kwargs['data']
        self.matrix_type = kwargs['matrix_type']
        super().__init__(**kwargs)
        
    def read(self):
        '''
        Overriding the read function - to return a list of Graph objects
        '''
        logger.info("Loading Pass Dataset.")
        
        data = self.data
        # Choosing the data with the required matrix type.
        data_mat = data[self.matrix_type]
        
        # Print Graph information
        logger.info(
            f"""node_features (x): {data_mat['x'][0].shape}
             \n adj_matrix (a): {data_mat['a'][0].shape}
             \n edge_features (e): {data_mat['e'][0].shape}
             """
        )

        # Print the content of 'binary' for the entire dataset
        y = data['binary']
        result_list = []

        for outer_list in y:
            for inner_list in outer_list:
                result_list.append(inner_list)
        y = result_list
        # Return a list of Graph objects
        return [
            Graph(x=x, a=a, e=e, y=y) for x, a, e, y in zip(
                data_mat['x'], data_mat['a'], data_mat['e'], data['binary']
            )
        ], y 


In [58]:
dataset,y = CounterDataset(data = data, matrix_type = adj_matrix.value)
dataset


Loading Pass Dataset.
node_features (x): (23, 12)
             
 adj_matrix (a): (23, 23)
             
 edge_features (e): (287, 6)
             


[Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph(n_nodes=23, n_node_features=12, n_edge_features=6, n_labels=1),
 Graph

In [98]:
for i in dataset[0:20]:
    print(min(i.x[:-1,8]),)

0.016880011504587202
0.013634746357180474
0.010659014312117967
0.009854207388079038
0.014909897069045403
0.005475159866671526
0.02348199407591292
0.011715394393582275
0.013836760780449847
0.00486714635638907
0.01165388985656409
0.009765005719651053
0.01923285855033085
0.013740484160952068
0.015138874474741867
0.012302471776758446
0.014120710451950333
0.009953851900808944
0.00946550529452847
0.00913274765040717


In [ ]:
N = max(g.n_nodes for g in dataset) # Number of nodes
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target
n = len(dataset)

In [ ]:

learning_rate = 1e-3  # Learning rate
epochs = 150  # Number of training epochs
batch_size = 16  # Batch size
channels = 128  # Hidden units for the neural network
layers = 3

In [ ]:
idxs = np.random.RandomState(seed=15).permutation(len(dataset))
split_va, split_te = int(0.7 * len(dataset)), int(0.69 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
dataset_tr = dataset[idx_tr]
dataset_te = dataset[idx_te]
loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1, shuffle = False)

# Display dataset information on target variable.
logger.info(f"n: {n}")
logger.info(f"Pct successful total: {round(np.asarray([graph.y[0] for graph in dataset]).sum() / n, 2)}")
logger.info(f"Pct successful train: {round(np.asarray([graph.y[0] for graph in dataset_tr]).sum() / (n * .7), 2)}")
logger.info(f"Pct successful test: {round(np.asarray([graph.y[0] for graph in dataset_te]).sum() / (n * .3), 2)}")


In [ ]:
import tensorflow as tf
import spektral as sp
from tensorflow.keras import layers, models
from spektral.data import BatchLoader

custom_learning_rate = 0.001  
custom_loss_fn = tf.keras.losses.BinaryCrossentropy() 
model = models.Sequential([
    layers.Input(shape=(23, 12,6)),  
    layers.Flatten(),
    layers.Dense(n_out, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=custom_learning_rate),
              loss=custom_loss_fn,
              metrics=['accuracy'])

In [ ]:
class CounterDataset2(Dataset):
    """
    Convert raw Graph data to a CounterDataset with Dataset class from the spektral library to include node features, 
    edge features and the adjacency matrix.
    """
    def __init__(self, **kwargs):
        '''
        Constructor to load parameters.
        '''
        self.data = kwargs['data']
        self.matrix_type = kwargs['matrix_type']
        super().__init__(**kwargs)
        
    def read(self):
        '''
        Overriding the read function - to return a list of Graph objects
        '''
        logger.info("Loading Pass Dataset.")
        
        data = self.data
        # Choosing the data with the required matrix type.
        data_mat = data[self.matrix_type]
        
        # Print Graph information
        logger.info(
            f"""node_features (x): {data_mat['x'][0].shape}
             \n adj_matrix (a): {data_mat['a'][0].shape}
             \n edge_features (e): {data_mat['e'][0].shape}
             """
        )
        
        # Return a list of Graph objects
        return [
            Graph(x=x, a=a, e=e, y=y) for x, a, e, y in zip(
                data_mat['x'], data_mat['a'], data_mat['e'], data['binary']
            )
        ]
dataset = CounterDataset2(data = data, matrix_type = adj_matrix.value)


In [ ]:
from spektral.data import DisjointLoader
loader_tr = DisjointLoader(dataset, batch_size=16,epochs=6)

#model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch)



In [ ]:
optimizer = Adam(learning_rate)
# Set up the logloss function
loss_fn = BinaryCrossentropy()

In [ ]:
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


# Print loss at each step of training.
step = loss = 0
for batch in loader_tr:
    step += 1
    loss += train_step(*batch)
    if step == loader_tr.steps_per_epoch:
        step = 0
        print("Loss: {}".format(loss / loader_tr.steps_per_epoch))
        loss = 0